# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정 

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir /content/drive/MyDrive/Datasets

mkdir: cannot create directory ‘/content/drive/MyDrive/Datasets’: File exists


### (2) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [3]:
import zipfile

In [4]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
# dataset_path  = '/content/drive/MyDrive/Datasets/'
dataset_path = '/content/drive/MyDrive/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [5]:
!mkdir /content/drive/MyDrive/Datasets/Car_Image_train; 

In [6]:
# 압축 해제
data = zipfile.ZipFile('/content/drive/MyDrive/Car_Images.zip')
data.extractall('/content/drive/MyDrive/Datasets/Car_Image_train')

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [7]:
# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()
!mkdir /content/drive/MyDrive/Datasets/Car_Image_test; 
!mkdir /content/drive/MyDrive/Datasets/Car_Image_test/normal
!mkdir /content/drive/MyDrive/Datasets/Car_Image_test/abnormal

# validation 폴더 만들기
!mkdir /content/drive/MyDrive/Datasets/Car_Image_val; 
!mkdir /content/drive/MyDrive/Datasets/Car_Image_val/normal
!mkdir /content/drive/MyDrive/Datasets/Car_Image_val/abnormal


## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [8]:
import random, shutil, os
import numpy as np

In [9]:
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Image_train/normal/'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Image_train/abnormal/'

In [10]:
# 전체 이미지 갯수를 확인합니다.
len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path))

(302, 303)

In [11]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(tr_n_path))*0.2), round(len(os.listdir(tr_ab_path))*0.2)]
print(te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
val_data_num = [ round((len(os.listdir(tr_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(tr_n_path))-te_data_num[1])*0.2) ]
print(val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(tr_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(tr_ab_path))- te_data_num[1] - val_data_num[1]]
print(train_data_num)

[60, 61]
[48, 48]
[194, 194]


In [12]:
train_n_set = os.listdir(tr_n_path)
train_ab_set = os.listdir(tr_ab_path)

#### 2) test 셋 추출

In [13]:
random.seed(2023)
te_n_path = '/content/drive/MyDrive/Datasets/Car_Image_test/normal/'
te_ab_path = '/content/drive/MyDrive/Datasets/Car_Image_test/abnormal/'

test_n_set = random.sample(os.listdir(tr_n_path), k = round(len(os.listdir(tr_n_path))*0.2))
test_ab_set = random.sample(os.listdir(tr_ab_path), k = round(len(os.listdir(tr_ab_path))*0.2))
for file in test_n_set :
    now_path = tr_n_path + file
    shutil.move(now_path, te_n_path)

for file in test_ab_set :
    now_path = tr_ab_path + file
    shutil.move(now_path, te_ab_path)

In [14]:
# 추출 후 이미지 갯수 확인

print(len(os.listdir(te_n_path)), len(os.listdir(te_ab_path)))
print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))

60 61
242 242


#### 3) validation 셋 추출

In [15]:
random.seed(2023)
val_n_path = '/content/drive/MyDrive/Datasets/Car_Image_val/normal/'
val_ab_path = '/content/drive/MyDrive/Datasets/Car_Image_val/abnormal/'

val_n_set = random.sample(os.listdir(tr_n_path), k = round(len(os.listdir(tr_n_path))*0.2))
val_ab_set = random.sample(os.listdir(tr_ab_path), k = round(len(os.listdir(tr_ab_path))*0.2))

for file in val_n_set :
    now_path = tr_n_path + file
    shutil.move(now_path, val_n_path)

for file in val_ab_set :
    now_path = tr_ab_path + file
    shutil.move(now_path, val_ab_path)



In [16]:
# 추출 후 이미지 갯수 확인
len(val_n_set), len(val_ab_set)

print(len(os.listdir(val_n_path)), len(os.listdir(val_ab_path)))
print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))

48 48
194 194


In [17]:
print(len(os.listdir(te_n_path)), len(os.listdir(te_ab_path)))
print(len(os.listdir(val_n_path)), len(os.listdir(val_ab_path)))
print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))

60 61
48 48
194 194


### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

* 복사하기 : shutil.copytree()

In [18]:
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Image_train/abnormal/', dataset_path+'copy_images/trainset/')
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Image_val/abnormal/', dataset_path+'copy_images/validset/')
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Image_test/abnormal/', dataset_path+'copy_images/testset/')

'/content/drive/MyDrive/Datasets/copy_images/testset/'

* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [19]:
trainset_path = '/content/drive/MyDrive/Datasets/copy_images/trainset/'
validset_path = '/content/drive/MyDrive/Datasets/copy_images/validset/'
testset_path = '/content/drive/MyDrive/Datasets/copy_images/testset/'

for file in os.listdir(trainset_path) :
    print(file)
    os.rename(trainset_path + file, trainset_path + 'ab_' + file)

for file in os.listdir(validset_path) :
    os.rename(validset_path + file, validset_path + 'ab_' + file)

for file in os.listdir(testset_path) :
    os.rename(testset_path + file, testset_path + 'ab_' + file)



DALLíñE 2023-03-10 18.51.26 - scratched car.png
DALLíñE 2023-03-10 18.51.29 - scratched car.png
DALLíñE 2023-03-10 18.51.32 - scratched car.png
DALLíñE 2023-03-10 18.53.06 - scratched car.png
DALLíñE 2023-03-10 18.53.08 - scratched car.png
DALLíñE 2023-03-10 18.53.58 - slightly damaged car.png
DALLíñE 2023-03-10 18.54.17 - slightly damaged car.png
DALLíñE 2023-03-10 18.54.19 - slightly damaged car.png
DALLíñE 2023-03-10 18.54.24 - slightly damaged car.png
DALLíñE 2023-03-10 22.04.36 - scratched car.png
DALLíñE 2023-03-10 22.04.39 - scratched car.png
DALLíñE 2023-03-10 22.04.42 - scratched car.png
DALLíñE 2023-03-10 22.37.56 - photo of a part of car.png
DALLíñE 2023-03-10 23.28.30 - photo of a part of car without blemish.png
DALLíñE 2023-03-10 23.34.52 - photo of a part of car without blemish.png
DALLíñE 2023-03-10 23.39.41 - photo of a part of car without blemish.png
DALLíñE 2023-03-10 23.55.00 - a part of car without blemish.png
DALLíñE 2023-03-10 2

#### 2) normal 파일 복사

In [20]:
nomal_train_path = '/content/drive/MyDrive/Datasets/Car_Image_train/normal/'
nomal_val_path = '/content/drive/MyDrive/Datasets/Car_Image_val/normal/'
nomal_test_path = '/content/drive/MyDrive/Datasets/Car_Image_test/normal/'

for file in os.listdir(nomal_train_path) : 
    shutil.copyfile(nomal_train_path+file , dataset_path+'copy_images/trainset/'+file)

for file in os.listdir(nomal_val_path) : 
    shutil.copyfile(nomal_val_path+file , dataset_path+'copy_images/validset/'+file)

for file in os.listdir(nomal_test_path) : 
    shutil.copyfile(nomal_test_path+file , dataset_path+'copy_images/testset/'+file)

* 데이터 갯수 조회

In [21]:
print(len(os.listdir(dataset_path+'copy_images/trainset/')))
print(len(os.listdir(dataset_path+'copy_images/validset/')))
print(len(os.listdir(dataset_path+'copy_images/testset/')))

388
96
121


In [22]:
print(len(os.listdir(tr_n_path)) + len(os.listdir(tr_ab_path)))
print(len(os.listdir(val_n_path)) + len(os.listdir(val_ab_path)))
print(len(os.listdir(te_n_path)) + len(os.listdir(te_ab_path)))

388
96
121


## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [24]:
# 이미지 목록 저장
img_train_list = os.listdir(dataset_path+'copy_images/trainset/')
img_valid_list = os.listdir(dataset_path+'copy_images/validset/')
img_test_list = os.listdir(dataset_path+'copy_images/testset/')

In [25]:
# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 224 ## 사이즈 조정 가능

In [26]:
from keras.utils import load_img, img_to_array

# x_train
x_train = []
train_path = dataset_path+'copy_images/trainset/'
for file in img_train_list :
    img_path = train_path + file
    img = load_img(img_path, target_size=(img_size, img_size))
    img_array = img_to_array(img)
    x_train.append(img_array)
x_train = np.array(x_train)

# x_valid
x_valid = []
val_path = dataset_path+'copy_images/validset/'
for file in img_valid_list :
    img_path = val_path + file
    img = load_img(img_path, target_size=(img_size, img_size))
    img_array = img_to_array(img)
    x_valid.append(img_array)
x_valid = np.array(x_valid)

# x_test

x_test = []
test_path = dataset_path+'copy_images/testset/'
for file in img_test_list :
    img_path = test_path + file
    img = load_img(img_path, target_size=(img_size, img_size))
    img_array = img_to_array(img)
    x_test.append(img_array)
x_test = np.array(x_test)

In [27]:
x_train.shape, x_valid.shape, x_test.shape

((388, 224, 224, 3), (96, 224, 224, 3), (121, 224, 224, 3))

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [28]:
###############################################

### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [29]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

388
194
---
96
48
---
121
61


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [30]:
y_train, y_valid, y_test = [0] * x_train.shape[0], [0] * x_valid.shape[0], [0] * x_test.shape[0]

# y_train
for idx, file in enumerate(img_train_list):
    if file.startswith('ab_') : y_train[idx] = 1
y_train = np.array(y_train)
# y_valid
for idx, file in enumerate(img_valid_list):
    if file.startswith('ab_') : y_valid[idx] = 1
y_valid = np.array(y_valid)
# 
for idx, file in enumerate(img_test_list):
    if file.startswith('ab_') : y_test[idx] = 1
y_test = np.array(y_test)

In [31]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape

((388, 224, 224, 3),
 (388,),
 (96, 224, 224, 3),
 (96,),
 (121, 224, 224, 3),
 (121,))

In [32]:
print(y_valid)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Min-Max Scaling

In [33]:
max_x = x_train.max() 
min_x = x_train.min()

x_train = (x_train - min_x)/(max_x - min_x)
x_valid = (x_valid - min_x)/(max_x - min_x)
x_test = (x_test - min_x)/(max_x - min_x)

## One-Hot Encoding

In [34]:
from keras.utils import to_categorical

class_n = len(np.unique(y_train))
y_train = to_categorical(y_train, class_n)
y_valid = to_categorical(y_valid, class_n)
y_test = to_categorical(y_test, class_n)

In [35]:
y_train.shape

(388, 2)

### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

## AlexNet 미니버전

# 3. 미니버젼 예시 :
    1. 처음 컨볼루션은 스트라이드1, 필터사이즈 5*5, 필터수 동일
    2. 맥스풀들은 전부 2*2
    3. 컨볼루션은 아래의 3X3 same. 13x13x384 까지(아래 맨 처음까지만)
    4. 3번에 이어서 바로 플래튼
    5. 덴스레이어는 순서대로 노드 384개, 192개 (2개만)
    6. 적절한 아웃풋레이어



In [37]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization

from sklearn.metrics import confusion_matrix, classification_report

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (227, 227, 3))
hl = Conv2D( filters = 96, kernel_size = (5, 5), strides = (1, 1), activation = 'relu')(il)
hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 256, kernel_size = (5, 5), padding = 'same', activation = 'relu')(hl)
hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 384, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = Flatten()(hl)

hl = Dense(384, activation = 'relu')(hl)
hl = Dense(192, activation = 'relu')(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.25)(hl)

ol = Dense(2, activation = 'sigmoid')(hl)

model_mini_alexnet = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_mini_alexnet.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_mini_alexnet.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 223, 223, 96)      7296      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 96)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 111, 111, 96)     384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 111, 111, 256)     614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 55, 55, 256)      0     

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 7,
                   restore_best_weights = True
                   )

In [ ]:
model_mini_alexnet.fit(x_train, y_train, callbacks = [es], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size=32)

Epoch 1/1000
13/13 [==============================] - 9s 749ms/step - loss: 0.4380 - accuracy: 0.8170 - val_loss: 4.2972 - val_accuracy: 0.4896
Epoch 2/1000
13/13 [==============================] - 6s 460ms/step - loss: 0.3558 - accuracy: 0.8686 - val_loss: 0.9985 - val_accuracy: 0.4896
Epoch 3/1000
13/13 [==============================] - 4s 306ms/step - loss: 0.3133 - accuracy: 0.8943 - val_loss: 1.1055 - val_accuracy: 0.5312
Epoch 4/1000
13/13 [==============================] - 6s 461ms/step - loss: 0.2717 - accuracy: 0.8995 - val_loss: 0.9094 - val_accuracy: 0.5312
Epoch 5/1000
13/13 [==============================] - 4s 308ms/step - loss: 0.2340 - accuracy: 0.9227 - val_loss: 1.4769 - val_accuracy: 0.5000
Epoch 6/1000
13/13 [==============================] - 4s 309ms/step - loss: 0.1971 - accuracy: 0.9330 - val_loss: 2.1251 - val_accuracy: 0.5000
Epoch 7/1000
13/13 [==============================] - 4s 310ms/step - loss: 0.1346 - accuracy: 0.9613 - val_loss: 3.0921 - val_accuracy:

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred = model_mini_alexnet.predict(x_test)

4/4 [==============================] - 2s 587ms/step


In [ ]:
single_y_pred = y_pred.argmax(axis=1)
single_y_test = y_test.argmax(axis=1)

In [ ]:
confusion_matrix(single_y_test, single_y_pred)

array([[ 3, 57],
       [ 0, 61]])

In [ ]:
print(classification_report(single_y_test, single_y_pred))

              precision    recall  f1-score   support

           0       1.00      0.05      0.10        60
           1       0.52      1.00      0.68        61

    accuracy                           0.53       121
   macro avg       0.76      0.53      0.39       121
weighted avg       0.76      0.53      0.39       121



### (4) 모델2
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

![알뤡스](https://cdn-images-1.medium.com/max/1600/1*jqKHgwZ8alM3K_JRYO_l4w.png)

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (227, 227, 3))
hl = Conv2D( filters = 96, kernel_size = (11, 11), strides = (4, 4), activation = 'relu')(il)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (5, 5), padding = 'same', activation = 'relu')(hl)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 768, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 768, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu')(hl)
hl = MaxPool2D( pool_size = (3, 3), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

# hl = keras.layers.GlobalAvgPool2D()(hl)
hl = Faltten()

hl = Dense(9216, activation = 'relu')(hl)
hl = Dense(4096, activation = 'relu')(hl)
hl = Dense(4096, activation = 'relu')(hl)

hl = Dropout(0.25)(hl)

ol = Dense(2, activation = 'sigmoid')(hl)

model_alexnet = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_alexnet.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_alexnet.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 512)       1229312   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 512)      0     

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 7,
                   restore_best_weights = True
                   )

In [ ]:
mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp2.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_alexnet.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 32)

Epoch 1/1000
13/13 [==============================] - ETA: 0s - loss: 0.4834 - accuracy: 0.7809
Epoch 1: val_loss improved from inf to 16.81968, saving model to /content/drive/MyDrive/Datasets/mcp2.h5
13/13 [==============================] - 8s 220ms/step - loss: 0.4834 - accuracy: 0.7809 - val_loss: 16.8197 - val_accuracy: 0.5000
Epoch 2/1000
12/13 [==========================>...] - ETA: 0s - loss: 0.3954 - accuracy: 0.8516
Epoch 2: val_loss improved from 16.81968 to 5.11182, saving model to /content/drive/MyDrive/Datasets/mcp2.h5
13/13 [==============================] - 2s 125ms/step - loss: 0.4032 - accuracy: 0.8454 - val_loss: 5.1118 - val_accuracy: 0.5000
Epoch 3/1000
12/13 [==========================>...] - ETA: 0s - loss: 0.3733 - accuracy: 0.8411
Epoch 3: val_loss improved from 5.11182 to 3.50659, saving model to /content/drive/MyDrive/Datasets/mcp2.h5
13/13 [==============================] - 2s 126ms/step - loss: 0.3736 - accuracy: 0.8428 - val_loss: 3.5066 - val_accuracy: 0.5

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred = model_alexnet.predict(x_test)

single_y_pred = y_pred.argmax(axis=1)
single_y_test = y_test.argmax(axis=1)

4/4 [==============================] - 1s 259ms/step


In [ ]:
confusion_matrix(single_y_test, single_y_pred)

array([[49, 11],
       [ 8, 53]])

In [ ]:
print(classification_report(single_y_test, single_y_pred))

              precision    recall  f1-score   support

           0       0.86      0.82      0.84        60
           1       0.83      0.87      0.85        61

    accuracy                           0.84       121
   macro avg       0.84      0.84      0.84       121
weighted avg       0.84      0.84      0.84       121



### (5) 모델3 - VGG-16(D구조)

![VGG-16(D구조)**굵은 텍스트**](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FK990l%2FbtqwDJ7C54R%2F664Ksm6gyTGBR1wK3YPDFk%2Fimg.png)

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [38]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (224, 224, 3))
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(il)
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

# hl = keras.layers.GlobalAvgPool2D()(hl)
hl = Flatten()(hl)

hl = Dense(4096, activation = 'relu')(hl)
hl = Dense(4096, activation = 'relu')(hl)
hl = Dense(1000, activation = 'relu')(hl)

hl = Dropout(0.25)(hl)

ol = Dense(2, activation = 'sigmoid')(hl)

model_vgg_16 = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_vgg_16.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_vgg_16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856 

In [39]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_vgg_16.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_vgg_16.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 32)

Epoch 1/1000
13/13 [==============================] - ETA: 0s - loss: 4.9880 - accuracy: 0.6082
Epoch 1: val_loss improved from inf to 159.24754, saving model to /content/drive/MyDrive/Datasets/mcp_vgg_16.h5
13/13 [==============================] - 51s 2s/step - loss: 4.9880 - accuracy: 0.6082 - val_loss: 159.2475 - val_accuracy: 0.5000
Epoch 2/1000
13/13 [==============================] - ETA: 0s - loss: 0.6430 - accuracy: 0.6959
Epoch 2: val_loss improved from 159.24754 to 102.99628, saving model to /content/drive/MyDrive/Datasets/mcp_vgg_16.h5
13/13 [==============================] - 13s 1s/step - loss: 0.6430 - accuracy: 0.6959 - val_loss: 102.9963 - val_accuracy: 0.5000
Epoch 3/1000
13/13 [==============================] - ETA: 0s - loss: 0.4806 - accuracy: 0.7912
Epoch 3: val_loss improved from 102.99628 to 22.43327, saving model to /content/drive/MyDrive/Datasets/mcp_vgg_16.h5
13/13 [==============================] - 13s 1s/step - loss: 0.4806 - accuracy: 0.7912 - val_loss: 22.4

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred = model_vgg_16.predict(x_test)

single_y_pred = y_pred.argmax(axis=1)
single_y_test = y_test.argmax(axis=1)

4/4 [==============================] - 1s 181ms/step


In [ ]:
confusion_matrix(single_y_test, single_y_pred)

array([[34, 26],
       [ 8, 53]])

In [ ]:
print(classification_report(single_y_test, single_y_pred))

              precision    recall  f1-score   support

           0       0.81      0.57      0.67        60
           1       0.67      0.87      0.76        61

    accuracy                           0.72       121
   macro avg       0.74      0.72      0.71       121
weighted avg       0.74      0.72      0.71       121



### (6) 모델4 - VGG-19(E구조)
![VGG-19](https://wikidocs.net/images/page/164796/vgg_Fig_07.jpeg)

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape = (224, 224, 3))
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(il)
hl = Conv2D( filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)
hl = Conv2D( filters = 512, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(hl)

hl = MaxPool2D( pool_size = (2, 2), strides = (2, 2))(hl)
hl = BatchNormalization()(hl)

hl = Flatten()(hl)

hl = Dense(4096, activation = 'relu')(hl)
hl = Dense(4096, activation = 'relu')(hl)
hl = Dense(1000, activation = 'relu')(hl)

hl = Dropout(0.25)(hl)

ol = Dense(2, activation = 'sigmoid')(hl)

model_vgg_19 = keras.models.Model(il, ol)


#  4. 모델 컴파일
model_vgg_19.compile(loss = 'binary_crossentropy', metrics = ['accuracy'],
              optimizer = 'adam')
# 모델 요약
model_vgg_19.summary()

ResourceExhaustedError: ignored

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   verbose = 1,
                   patience = 15,
                   restore_best_weights = True
                   )

mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Datasets/mcp_vgg_19.h5',
                      monitor = 'val_loss',
                      verbose = 1,
                      save_best_only = True,
                      save_weights_only = False
                      )

In [ ]:
model_vgg_19.fit(x_train, y_train, callbacks = [es, mcp], epochs = 1000, validation_data = (x_valid, y_valid), verbose = 1, batch_size = 32)

Epoch 1/1000


ResourceExhaustedError: ignored

## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 280 ## 사이즈 조정 가능

train_path = dataset_path+'Car_Images_train/'
valid_path = dataset_path+'Car_Images_valid/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
train_datagen = 

valid_datagen = 


#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = 

valid_generator = 


#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [ ]:
from tensorflow.keras.applications import VGG16

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
base_model = VGG16(                 )




#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

#### 4) 성능 평가